In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from qiskit_metal import MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround

In [ ]:
%metal_heading Making your design in Metal

In [ ]:
design = designs.MultiPlanar({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [ ]:
design.delete_all_components()

In [ ]:
# Making the qubit
q_opts = dict(
    pos_x = "2mm",
    pos_y = "2mm",
    connection_pads = dict(
        readout = dict()
    )
)

q1 = TransmonPocket(design, "Q1", options=q_opts)

gui.rebuild()
gui.autoscale()

In [ ]:
# Making the open to ground
otg_opts = dict(
    pos_x = "4mm",
    pos_y = "2mm",
)

otg1 = OpenToGround(design, "otg1", options=otg_opts)

gui.rebuild()
gui.autoscale()

In [ ]:
meander_opts1 = dict(
    pin_inputs = dict(
        start_pin = dict(component = "Q1", pin = "readout"),
        end_pin = dict(component = "otg1", pin = "open")
    ),
    lead = dict(
        start_straight = "100um"
    ),
    fillet = "99.99um",
    asymmetry = "100um",
)

mtl1 = RouteMeander(design, "mtl1", options=meander_opts1)

gui.rebuild()
gui.autoscale()

In [ ]:
%metal_heading Render to Gmsh

In [ ]:
from qiskit_metal.renderers.renderer_gmsh.gmsh_renderer import QGmshRenderer

# Instantiate QGmshRenderer
gmsh_renderer = QGmshRenderer(design)

# Set necessary options for mesh generation
gmsh_renderer.options.mesh.min_size = "5um"
gmsh_renderer.options.mesh.max_size = "70um"

In [ ]:
# Render the wireframe of the design
gmsh_renderer.render_design(mesh_geoms=False, skip_junctions=True)

# Launch Gmsh GUI to verify
gmsh_renderer.launch_gui()

In [ ]:
# Once you've verified that the design looks correct
gmsh_renderer.add_mesh(dim=3, intelli_mesh=False)
gmsh_renderer.launch_gui()

In [ ]:
# Close and destroy the renderer object
gmsh_renderer.close()

In [ ]:
%metal_heading Intelli-mesh in Gmsh

In [ ]:
gmsh_renderer = QGmshRenderer(design)
gmsh_renderer.options.mesh.min_size = "5um"
gmsh_renderer.options.mesh.max_size = "70um"

gmsh_renderer.render_design(mesh_geoms=False)
gmsh_renderer.add_mesh(dim=3, intelli_mesh=True)

gmsh_renderer.launch_gui()

In [ ]:
# Export the mesh to a file
gmsh_renderer.export_mesh("test.msh")

In [ ]:
# Close and destroy the renderer object
gmsh_renderer.close()

In [ ]:
gui.main_window.close()